# Chapter 4. Composite Types (II)
*    **Structs**
     *    Struct Literals
     *    Comparing Structs
     *    Struct Embedding and Anonymous Fields
*    JSON
*    Text and HTML Templates

## 4.4. Structs

* A struct is an aggregate data type grouping together zero or more named values of arbitrar types as a single entity
* Each value is called a field
* The name of a struct field is exported if it begins with a capital letter; this is Go's main access control mechanism
* A struct type may contain a mixture of exported and unexported fields

In [1]:
import "time"
type Employee struct {
    ID int
    Name string
    Address string
    DoB time.Time
    Position string
    Salary int
    ManagerID int
}
employeeDB := make(map[int]*Employee)
alan := Employee{ID:1, ManagerID: 3, Position: "Vice President"}
dilbert := Employee{ID:2, ManagerID: 1}
employeeDB[alan.ID] = &alan
employeeDB[dilbert.ID] = &dilbert
dilbert.Salary -= 5000
position := &dilbert.Position
*position = "Senior" + *position
var employeeOfTheMonth *Employee = &dilbert
//employeeOfTheMonth.Position += " (proactive team player)"
(*employeeOfTheMonth).Position += " (proactive team player)"

In [2]:
import "fmt"
func EmployeeByID(id int) *Employee { return employeeDB[id]}
fmt.Println(EmployeeByID(dilbert.ManagerID).Position)
id := dilbert.ID
EmployeeByID(id).Salary = 0

Vice President


In [6]:
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 101.

// Package treesort provides insertion sort using an unbalanced binary tree.
//package treesort

//!+
import (
	"math/rand"
	"sort"
    "fmt"
)
type tree struct {
	value       int
	left, right *tree
}

// Sort sorts values in place.
func Sort(values []int) {
	var root *tree
	for _, v := range values {
		root = add(root, v)
	}
	//appendValues(values[:0], root)
}

// appendValues appends the elements of t to values in order
// and returns the resulting slice.
func appendValues(values []int, t *tree) []int {
	if t != nil {
		values = appendValues(values, t.left)
		values = append(values, t.value)
		values = appendValues(values, t.right)
	}
	return values
}

func add(t *tree, value int) *tree {
	if t == nil {
		// Equivalent to return &tree{value: value}.
		t = new(tree)
		t.value = value
		return t
	}
	if value < t.value {
		t.left = add(t.left, value)
	} else {
		//t.right = add(t.right, value)
	}
	return t
}

data := make([]int, 50)
for i := range data {
	data[i] = rand.Int() % 50
}
    
	//treesort.Sort(data)
//Sort(data)
//sort.Ints(data)
if !sort.IntsAreSorted(data) {
		fmt.Printf("not sorted: %v", data)
} else {
    fmt.Printf("sorted: %v", data)
}
//!-

not sorted: [45 5 12 16 37 33 40 37 28 12 23 14 23 47 27 11 29 12 15 41 49 28 23 6 12 17 44 7 27 37 18 44 3 39 7 9 1 49 48 29 1 25 3 43 38 24 43 11 12 31]

<pre>
<code>
$go get gopl.io/ch4
$go build gopl.io/ch4/treesort
$go install gopl.io/ch4/treesort
$go test gopl.io/ch4/treesort
</code>
</pre>

### 4.4.1. Struct Literals

* A value of a struct type can be written using a struct literal specifying values for its fields
* There are two forms of struct literal
* The first form requires a value be specified for every field, in the right order
* The second form is used, in which a struct value is initialized by listing some or all of the field names and their corresponding values
* The two forms cannot be mixed in the same literal

In [21]:
type Point struct{ X, Y int}
p := Point{1,2} // the first form
q := Point{X:1, Y:2} // the second form

In [46]:
type T struct{ a, b int} // a and b are not exported
var _ = T{a:1, b:2}

### 4.4.3. Struct Embedding and Anonymous Fields

In [47]:
type Circle struct {
    X,Y, Radius int
}
type Wheel struct {
    X,Y, Radius, Spokes int
}
var w Wheel
w.X = 8
w.Y = 8
w.Radius = 5
w.Spokes = 20

* Factor out their common parts

In [56]:
type Point struct {
    X, Y int
}
type Circle struct {
    Center Point
    Radius int
}
type Wheel struct {
    Circle Circle
    Spokes int
}
var w Wheel
w.Circle.Center.X = 8
w.Circle.Center.Y = 8
w.Circle.Radius = 5
w.Spokes = 20
fmt.Println(w)

{{{8 8} 5} 20}


15 <nil>

* Declare a field with a type but no name
* Such fields are called anonymous fields
* Due to embedding, we can refer to the names at the leaves of the implicit tree without giving the intervening names

In [55]:
type Circle struct {
    Point
    Radius int
}
type Wheel struct {
    Circle 
    Spokes int
}
var w Wheel
w.X = 8    //w.Circle.Point.X
w.Y = 8    //w.Circle.Point.Y
w.Radius = 5 //w.Circle.Radius
w.Spokes = 20 
fmt.Println(w)

{{{8 8} 5} 20}


15 <nil>

* There's no corresponding shorthand for the struct literal syntax

In [58]:
w = Wheel{8,8,5,20} 

ERROR: cannot convert untyped constant {int 8} to <main.Circle>

In [60]:
w = Wheel{X:8, Y:8, Radius: 5, Spokes: 20}

ERROR: repl.go:1:5: unknown field 'X' in struct literal of type main.Wheel

In [73]:
type Point struct {
    X, Y int
}
type Circle struct {
    Point
    Radius int
}
type Wheel struct {
    Circle 
    Spokes int
}
var w Wheel
w = Wheel{Circle{Point{8, 8}, 5}, 20}

w = Wheel{
		Circle: Circle{
			Point:  Point{X: 8, Y: 8},
			Radius: 5,
		},
		Spokes: 20, // NOTE: trailing comma necessary here (and at Radius)
}

//fmt.Printf("%#v\n", w)
fmt.Printf("%+v\n",w)
	// Output:
	// Wheel{Circle:Circle{Point:Point{X:8, Y:8}, Radius:5}, Spokes:20}

w.X = 42

//fmt.Printf("%#v\n", w)
fmt.Printf("%+v\n", w)
	// Output:
	// Wheel{Circle:Circle{Point:Point{X:42, Y:8}, Radius:5}, Spokes:20}
	//!-

{Circle:{Point:{X:8 Y:8} Radius:5} Spokes:20}
{Circle:{Point:{X:42 Y:8} Radius:5} Spokes:20}


47 <nil>

## 4.5. JSON
* JavaScript Object Notation (JSON) is a standard notation for sending and receiving structured information
* Other similar notation: XML, ASN.1, and Google's Protocol Buffers
* API: encoding/json, encoding/xml, encoding/asn1
* basic JSON types are numbers(decimal or scientic notation), booleans(true, false), and strings (sequene of Unicode code points)
* basic JSON types can be combined recursively using JSON arrays and objects
* JSON array is an ordered sequence of values, written as a comma-separated list enclosed in square brackets
* JSON array are used to encode Go arrays and slices
* JSON object is a mapping from strings to values, written as a sequence of name:value pairs separated by commas and surrounded by braces
* JSON objects are used to encode GO maps and structs

Example

type   |value
-------|-----------------------------------
boolean|true
number | -273.15
string |"She said \"Hello, 世界\""
array  | ["gold", "silver", "bronze"]
object |{"year": 1980,<br>"event": "archery",<br>"medals":["gold","silver","brone"]} 

In [34]:
import ("encoding/json";"fmt";"log")

type Movie struct {
	Title  string
	Year   int  `json:"released"`
	Color  bool `json:"color,omitempty"`
	Actors []string
}

var movies = []Movie{
	{Title: "Casablanca", Year: 1942, Color: false,
		Actors: []string{"Humphrey Bogart", "Ingrid Bergman"}},
	{Title: "Cool Hand Luke", Year: 1967, Color: true,
		Actors: []string{"Paul Newman"}},
	{Title: "Bullitt", Year: 1968, Color: true,
		Actors: []string{"Steve McQueen", "Jacqueline Bisset"}},
	// ...
}

In [35]:
		//!+Marshal
		data, err := json.Marshal(movies)
		if err != nil {
			log.Fatalf("JSON marshaling failed: %s", err)
		}
		fmt.Printf("%s\n", data)
		//!-Marshal

[{"Title":"Casablanca","released":1942,"Actors":["Humphrey Bogart","Ingrid Bergman"]},{"Title":"Cool Hand Luke","released":1967,"color":true,"Actors":["Paul Newman"]},{"Title":"Bullitt","released":1968,"color":true,"Actors":["Steve McQueen","Jacqueline Bisset"]}]


264 <nil>

In [36]:
	{
		//!+MarshalIndent
		data, err := json.MarshalIndent(movies, "", "    ")
		if err != nil {
			log.Fatalf("JSON marshaling failed: %s", err)
		}
		fmt.Printf("%s\n", data)
		//!-MarshalIndent

		//!+Unmarshal
		var titles []struct{ Title string }
		if err := json.Unmarshal(data, &titles); err != nil {
			log.Fatalf("JSON unmarshaling failed: %s", err)
		}
		fmt.Println(titles) // "[{Casablanca} {Cool Hand Luke} {Bullitt}]"
		//!-Unmarshal
	}

[
    {
        "Title": "Casablanca",
        "released": 1942,
        "Actors": [
            "Humphrey Bogart",
            "Ingrid Bergman"
        ]
    },
    {
        "Title": "Cool Hand Luke",
        "released": 1967,
        "color": true,
        "Actors": [
            "Paul Newman"
        ]
    },
    {
        "Title": "Bullitt",
        "released": 1968,
        "color": true,
        "Actors": [
            "Steve McQueen",
            "Jacqueline Bisset"
        ]
    }
]
[{Casablanca} {Cool Hand Luke} {Bullitt}]


In [75]:
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 108.

// Movie prints Movies as JSON.
//package main

import (
	"encoding/json"
	"fmt"
	"log"
)

//!+
type Movie struct {
	Title  string
	Year   int  `json:"released"`
	Color  bool `json:"color,omitempty"`
	Actors []string
}

var movies = []Movie{
	{Title: "Casablanca", Year: 1942, Color: false,
		Actors: []string{"Humphrey Bogart", "Ingrid Bergman"}},
	{Title: "Cool Hand Luke", Year: 1967, Color: true,
		Actors: []string{"Paul Newman"}},
	{Title: "Bullitt", Year: 1968, Color: true,
		Actors: []string{"Steve McQueen", "Jacqueline Bisset"}},
	// ...
}

//!-

		//!+Marshal
		data, err := json.Marshal(movies)
		if err != nil {
			log.Fatalf("JSON marshaling failed: %s", err)
		}
		fmt.Printf("%s\n", data)
		//!-Marshal


	{
		//!+MarshalIndent
		data, err := json.MarshalIndent(movies, "", "    ")
		if err != nil {
			log.Fatalf("JSON marshaling failed: %s", err)
		}
		fmt.Printf("%s\n", data)
		//!-MarshalIndent

		//!+Unmarshal
		var titles []struct{ Title string }
		if err := json.Unmarshal(data, &titles); err != nil {
			log.Fatalf("JSON unmarshaling failed: %s", err)
		}
		fmt.Println(titles) // "[{Casablanca} {Cool Hand Luke} {Bullitt}]"
		//!-Unmarshal
	}

/*
//!+output
[{"Title":"Casablanca","released":1942,"Actors":["Humphrey Bogart","Ingr
id Bergman"]},{"Title":"Cool Hand Luke","released":1967,"color":true,"Ac
tors":["Paul Newman"]},{"Title":"Bullitt","released":1968,"color":true,"
Actors":["Steve McQueen","Jacqueline Bisset"]}]
//!-output
*/

/*
//!+indented
[
    {
        "Title": "Casablanca",
        "released": 1942,
        "Actors": [
            "Humphrey Bogart",
            "Ingrid Bergman"
        ]
    },
    {
        "Title": "Cool Hand Luke",
        "released": 1967,
        "color": true,
        "Actors": [
            "Paul Newman"
        ]
    },
    {
        "Title": "Bullitt",
        "released": 1968,
        "color": true,
        "Actors": [
            "Steve McQueen",
            "Jacqueline Bisset"
        ]
    }
]
//!-indented
*/

[{"Title":"Casablanca","released":1942,"Actors":["Humphrey Bogart","Ingrid Bergman"]},{"Title":"Cool Hand Luke","released":1967,"color":true,"Actors":["Paul Newman"]},{"Title":"Bullitt","released":1968,"color":true,"Actors":["Steve McQueen","Jacqueline Bisset"]}]
[
    {
        "Title": "Casablanca",
        "released": 1942,
        "Actors": [
            "Humphrey Bogart",
            "Ingrid Bergman"
        ]
    },
    {
        "Title": "Cool Hand Luke",
        "released": 1967,
        "color": true,
        "Actors": [
            "Paul Newman"
        ]
    },
    {
        "Title": "Bullitt",
        "released": 1968,
        "color": true,
        "Actors": [
            "Steve McQueen",
            "Jacqueline Bisset"
        ]
    }
]
[{Casablanca} {Cool Hand Luke} {Bullitt}]


In [37]:
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 110.
//!+

// Package github provides a Go API for the GitHub issue tracker.
// See https://developer.github.com/v3/search/#search-issues.
//package github

import "time"

const IssuesURL = "https://api.github.com/search/issues"

type IssuesSearchResult struct {
	TotalCount int `json:"total_count"`
	Items      []*Issue
}

type Issue struct {
	Number    int
	HTMLURL   string `json:"html_url"`
	Title     string
	State     string
	User      *User
	CreatedAt time.Time `json:"created_at"`
	Body      string    // in Markdown format
}

type User struct {
	Login   string
	HTMLURL string `json:"html_url"`
}

//!-

In [38]:
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

//!+

//package github

import (
	"encoding/json"
	"fmt"
	"net/http"
	"net/url"
	"strings"
)

// SearchIssues queries the GitHub issue tracker.
func SearchIssues(terms []string) (*IssuesSearchResult, error) {
	q := url.QueryEscape(strings.Join(terms, " "))
	resp, err := http.Get(IssuesURL + "?q=" + q)
	if err != nil {
		return nil, err
	}
	//!-
	// For long-term stability, instead of http.Get, use the
	// variant below which adds an HTTP request header indicating
	// that only version 3 of the GitHub API is acceptable.
	//
	//   req, err := http.NewRequest("GET", IssuesURL+"?q="+q, nil)
	//   if err != nil {
	//       return nil, err
	//   }
	//   req.Header.Set(
	//       "Accept", "application/vnd.github.v3.text-match+json")
	//   resp, err := http.DefaultClient.Do(req)
	//!+

	// We must close resp.Body on all execution paths.
	// (Chapter 5 presents 'defer', which makes this simpler.)
	if resp.StatusCode != http.StatusOK {
		resp.Body.Close()
		return nil, fmt.Errorf("search query failed: %s", resp.Status)
	}

	var result IssuesSearchResult
	if err := json.NewDecoder(resp.Body).Decode(&result); err != nil {
		resp.Body.Close()
		return nil, err
	}
	resp.Body.Close()
	return &result, nil
}

//!-

In [39]:
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 112.
//!+

// Issues prints a table of GitHub issues matching the search terms.
//package main

import (
	"fmt"
	"log"
	"os"

	"gopl.io/ch4/github"
)

//!+
func main() {
	//result, err := github.SearchIssues(os.Args[1:])
    t := []string{"repo:golang/go","is:open","json","decoder"}
    result, err := github.SearchIssues(t)
	if err != nil {
		log.Fatal(err)
	}
	fmt.Printf("%d issues:\n", result.TotalCount)
	for _, item := range result.Items {
		fmt.Printf("#%-5d %9.9s %.55s\n",
			item.Number, item.User.Login, item.Title)
	}
}

//!-

/*
//!+textoutput
$ go build gopl.io/ch4/issues
$ ./issues repo:golang/go is:open json decoder
13 issues:
#5680    eaigner encoding/json: set key converter on en/decoder
#6050  gopherbot encoding/json: provide tokenizer
#8658  gopherbot encoding/json: use bufio
#8462  kortschak encoding/json: UnmarshalText confuses json.Unmarshal
#5901        rsc encoding/json: allow override type marshaling
#9812  klauspost encoding/json: string tag not symmetric
#7872  extempora encoding/json: Encoder internally buffers full output
#9650    cespare encoding/json: Decoding gives errPhase when unmarshalin
#6716  gopherbot encoding/json: include field name in unmarshal error me
#6901  lukescott encoding/json, encoding/xml: option to treat unknown fi
#6384    joeshaw encoding/json: encode precise floating point integers u
#6647    btracey x/tools/cmd/godoc: display type kind of each named type
#4237  gjemiller encoding/base64: URLEncoding padding is optional
//!-textoutput
*/

In [40]:
os.Args = []string{"./issues", "repo:golang/go", "is:open", "json", "decoder"}
main()

42 issues:
#33416   bserdar encoding/json: This CL adds Decoder.InternKeys
#34647 babolivie encoding/json: fix byte counter increments when using d
#34543  maxatome encoding/json: Unmarshal & json.(*Decoder).Token report
#32779       rsc proposal: encoding/json: memoize strings during decode?
#34564  mdempsky go/internal/gcimporter: single source of truth for deco
#28923     mvdan encoding/json: speed up the decoding scanner
#11046     kurin encoding/json: Decoder internally buffers full input
#33854     Qhesz encoding/json: unmarshal option to treat omitted fields
#30301     zelch encoding/xml: option to treat unknown fields as an erro
#26946    deuill encoding/json: clarify what happens when unmarshaling i
#33835     Qhesz encoding/json: unmarshalling null into non-nullable gol
#12001 lukescott encoding/json: Marshaler/Unmarshaler not stream friendl
#31789  mgritter encoding/json: provide a way to limit recursion depth
#29035    jaswdr proposal: encoding/json: add error var to compar

## 4.6. Text and HTML Templates
* the simplest formatting: Printf
* separate the format from the code 
* **text/template** and **html/template** packages provide a mechanism for substituting the values of variables into a text or HTML template 


* template is a string or file containing one or more portions enclosed in double braces, {{...}}, called actions

In [4]:
import "fmt"
const templ = `{{.TotalCount}} issues:
{{range .Items}}----------------------------------------
Number: {{.Number}}
User:   {{.User.Login}}
Title:  {{.Title | printf "%.64s"}}
Age:    {{.CreatedAt | daysAgo}} days
{{end}}`
fmt.Println(templ)

{{.TotalCount}} issues:
{{range .Items}}----------------------------------------
Number: {{.Number}}
User:   {{.User.Login}}
Title:  {{.Title | printf "%.64s"}}
Age:    {{.CreatedAt | daysAgo}} days
{{end}}


207 <nil>

In [12]:
import ("time";"text/template";"os")
func daysAgo(t time.Time) int {
    return int(time.Since(t).Hours()/24)
}
report, err := template.New("report").
   Funcs(template.FuncMap{"daysAgo": daysAgo}).
   Parse(templ)
if (err != nil) {
    fmt.Fprintln(os.Stderr,"error")
} else {
    fmt.Printf("%q\n",report)
}

&{"report" %!q(*parse.Tree=&{report report 0xc000482f00 {{.TotalCount}} issues:
{{range .Items}}----------------------------------------
Number: {{.Number}}
User:   {{.User.Login}}
Title:  {{.Title | printf "%.64s"}}
Age:    {{.CreatedAt | daysAgo}} days
{{end}} [] <nil> [{7 206  7} {10 24 {{ 2} {0 0  0}] 1 [] map[]}) %!q(*template.common=&{map[report:0xc000421fc0] {0} {{0 0} 0 0 0 0} map[daysAgo:0xa98950] map[daysAgo:{0xc000439d40 0xc00053b410 19}]}) "" ""}


In [1]:
import ("log";"os";"text/template";"time";"gopl.io/ch4/github")
const templ = `{{.TotalCount}} issues:
{{range .Items}}----------------------------------------
Number: {{.Number}}
User:   {{.User.Login}}
Title:  {{.Title | printf "%.64s"}}
Age:    {{.CreatedAt | daysAgo}} days
{{end}}`

func daysAgo(t time.Time) int {
	return int(time.Since(t).Hours() / 24)
}

var report = template.Must(template.New("issuelist").
	Funcs(template.FuncMap{"daysAgo": daysAgo}).
	Parse(templ))

func main() {
	result, err := github.SearchIssues(os.Args[1:])
	if err != nil {
		log.Fatal(err)
	}
	if err := report.Execute(os.Stdout, result); err != nil {
		log.Fatal(err)
	}
}

//!-exec

func noMust() {
	//!+parse
	report, err := template.New("report").
		Funcs(template.FuncMap{"daysAgo": daysAgo}).
		Parse(templ)
	if err != nil {
		log.Fatal(err)
	}
	//!-parse
	result, err := github.SearchIssues(os.Args[1:])
	if err != nil {
		log.Fatal(err)
	}
	if err := report.Execute(os.Stdout, result); err != nil {
		log.Fatal(err)
	}
}

/*
//!+output
$ go build gopl.io/ch4/issuesreport
$ ./issuesreport repo:golang/go is:open json decoder
13 issues:
----------------------------------------
Number: 5680
User:   eaigner
Title:  encoding/json: set key converter on en/decoder
Age:    750 days
----------------------------------------
Number: 6050
User:   gopherbot
Title:  encoding/json: provide tokenizer
Age:    695 days
----------------------------------------
...
//!-output
*/

In [2]:
os.Args = []string{"issuereport", "repo:golang/go","is:open","json","decoder"}
main()

42 issues:
----------------------------------------
Number: 33416
User:   bserdar
Title:  encoding/json: This CL adds Decoder.InternKeys
Age:    79 days
----------------------------------------
Number: 34647
User:   babolivier
Title:  encoding/json: fix byte counter increments when using decoder.To
Age:    18 days
----------------------------------------
Number: 34543
User:   maxatome
Title:  encoding/json: Unmarshal & json.(*Decoder).Token report differen
Age:    24 days
----------------------------------------
Number: 32779
User:   rsc
Title:  proposal: encoding/json: memoize strings during decode?
Age:    116 days
----------------------------------------
Number: 34564
User:   mdempsky
Title:  go/internal/gcimporter: single source of truth for decoder logic
Age:    23 days
----------------------------------------
Number: 28923
User:   mvdan
Title:  encoding/json: speed up the decoding scanner
Age:    332 days
----------------------------------------
Number: 11046
User:   kurin
Title:

* html/template use the same API and expression language as text/template
* with HTML, JavaScript, CSS, or URLs

In [32]:
import ("log";"os";"gopl.io/ch4/github")

//!+template
import "html/template"

var issueList = template.Must(template.New("issuelist").Parse(`
<h1>{{.TotalCount}} issues</h1>
<table>
<tr style='text-align: left'>
  <th>#</th>
  <th>State</th>
  <th>User</th>
  <th>Title</th>
</tr>
{{range .Items}}
<tr>
  <td><a href='{{.HTMLURL}}'>{{.Number}}</a></td>
  <td>{{.State}}</td>
  <td><a href='{{.User.HTMLURL}}'>{{.User.Login}}</a></td>
  <td><a href='{{.HTMLURL}}'>{{.Title}}</a></td>
</tr>
{{end}}
</table>
`))

//!-template

//!+
func main() {
	result, err := github.SearchIssues(os.Args[1:])
	if err != nil {
		log.Fatal(err)
	}
	if err := issueList.Execute(os.Stdout, result); err != nil {
		log.Fatal(err)
	}
}

//!-

In [3]:
os.Args = []string{"issuehtml","repo:golang/go","commenter:gopherbot","json","encoder"}
main()


<h1>37 issues</h1>
<table>
<tr style='text-align: left'>
  <th>#</th>
  <th>State</th>
  <th>User</th>
  <th>Title</th>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/7872'>7872</a></td>
  <td>open</td>
  <td><a href='https://github.com/extemporalgenome'>extemporalgenome</a></td>
  <td><a href='https://github.com/golang/go/issues/7872'>encoding/json: Encoder internally buffers full output</a></td>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/27735'>27735</a></td>
  <td>open</td>
  <td><a href='https://github.com/dsnet'>dsnet</a></td>
  <td><a href='https://github.com/golang/go/issues/27735'>encoding/json: incorrect usage of sync.Pool</a></td>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/10769'>10769</a></td>
  <td>open</td>
  <td><a href='https://github.com/bep'>bep</a></td>
  <td><a href='https://github.com/golang/go/issues/10769'>encoding/json: detect circular data structures when encoding</a></td>
</tr>

<tr>
  <td><a href='https:


<tr>
  <td><a href='https://github.com/golang/go/issues/6901'>6901</a></td>
  <td>closed</td>
  <td><a href='https://github.com/lukescott'>lukescott</a></td>
  <td><a href='https://github.com/golang/go/issues/6901'>encoding/json, encoding/xml: option to treat unknown fields as an error</a></td>
</tr>

</table>


[issues.html](http://skhuang.github.io/ch4/issues.html)

In [33]:
os.Args = []string{"issuehtml","repo:golang/go","3133","10535"}
main()


<h1>2 issues</h1>
<table>
<tr style='text-align: left'>
  <th>#</th>
  <th>State</th>
  <th>User</th>
  <th>Title</th>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/10535'>10535</a></td>
  <td>open</td>
  <td><a href='https://github.com/dvyukov'>dvyukov</a></td>
  <td><a href='https://github.com/golang/go/issues/10535'>x/net/html: void element &lt;link&gt; has child nodes</a></td>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/3133'>3133</a></td>
  <td>closed</td>
  <td><a href='https://github.com/ukai'>ukai</a></td>
  <td><a href='https://github.com/golang/go/issues/3133'>html/template: escape xmldesc as &amp;lt;?xml</a></td>
</tr>

</table>


[issue2.html](https://skhuang.github.io/ch4/issues2.html)

In [41]:
import ("fmt";"html/template";"log";"os")

//!+
func main() {
	const templ = `<p>A: {{.A}}</p><p>B: {{.B}}</p>`
	t := template.Must(template.New("escape").Parse(templ))
	var data struct {
		A string        // untrusted plain text
		B template.HTML // trusted HTML
	}
    fmt.Printf("%T\n",data)
    data.A = "<b>Hello!</b>"
    data.B = "<b>Hello!</b>"
    var  b template.HTML= template.HTML("test")
	if err := t.Execute(os.Stdout, data); err != nil {
		log.Fatal(err)
	}
}

//main()
//!-

ERROR: repl.go:13:5: error compiling assignment: data.B = "<b>Hello!</b>"
	reflect.Value.Convert: value of type reflect.Value cannot be converted to type template.HTML

[autoescape.html](http://skhuang.github.io/ch4/autoescape.html)